In [1]:
!pip install nbconvert

In [3]:
# -*- coding: utf-8 -*-
"""
PPO Dynamic Pricing with Gymnasium + RPT Integrated
Tích hợp đầy đủ: RPT Demand → PPO Pricing → Giá đề xuất
Không phụ thuộc file rpt.py / news_embeding.py
"""

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pickle
import json
import pandas as pd
from datetime import datetime, timedelta
import os
import warnings
import gymnasium as gym
from gymnasium import spaces
!git clone https://github.com/FongNgoo/Basic_Dynamic_Prices_base_on_Demand_Model.git
%cd Basic_Dynamic_Prices_base_on_Demand_Model
warnings.filterwarnings("ignore")

Cloning into 'Basic_Dynamic_Prices_base_on_Demand_Model'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 152 (delta 22), reused 152 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (152/152), 7.72 MiB | 29.70 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/Basic_Dynamic_Prices_base_on_Demand_Model


# Config Set Up

In [4]:
RPT_MODEL_PATH = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model/Output/rpt_demand_best.pth"
PREPROCESSED_NPZ = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model/Output/preprocessed_data.npz"
SCALERS_PKL = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model/Output/scalers.pkl"
PPO_MODEL_SAVE = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model/Output/ppo_pricing_best.pth"

## PPO hyperparameters

In [5]:
STATE_DIM = 3 + 8 + 3 + 45 + 11  # demand + env + available + news + rev_hist(10) + current_rev
ACTION_DIM = 3
HIDDEN_DIM = 256
PPO_EPOCHS = 4
CLIP_EPS = 0.2
GAMMA = 0.99
LAMBDA = 0.95
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
MAX_GRAD_NORM = 0.5
PARITY_PENALTY = 1e7  # Phạt cực nặng nếu vi phạm

## Price Constraints



In [6]:
MIN_PRICE = {'single': 800000, 'double': 1200000, 'vip': 2500000}
MAX_PRICE = {'single': 3000000, 'double': 5000000, 'vip': 10000000}

PRICE_PARITY = {
    'double_single_ratio': (1.3, 1.8),
    'vip_single_ratio': (2.0, 3.0)
}

WINDOW_SIZE = 60  # Đã dùng đúng trong sliding window

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Load scalers
with open(SCALERS_PKL, 'rb') as f:
    scalers = pickle.load(f)
price_scaler = scalers['price']

# Import RPT
!jupyter nbconvert --to python RPT.ipynb
import RPT

print("IMPORT QUA GIT THÀNH CÔNG!")

[NbConvertApp] Converting notebook RPT.ipynb to python
[NbConvertApp] Writing 10706 bytes to RPT.py
[AUTO] h = 144
   → h % (R*3) = 0 = 0
   → h % num_heads = 0 = 0
[AUTO] h = 144 → per_room: 48, per_kernel: 16, head_dim: 18


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab_Notebooks/AI_for_Business_Project/Output/preprocessed_data.npz'

In [8]:
%cd /content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model
!git add .
!git commit -m "Cập nhật mới từ Colab"
!git push origin main

/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@a2ef3a18cebb.(none)')
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
